# Test Rechunker on NWM

Save a small piece of NWM and then rechunk it using the `rechunker` library.

In [21]:
import json
from os.path import join
from os import makedirs
import os
import shutil

import dask.dataframe as dd
import psycopg2
import shapely
import shapely.wkt
import geopandas as gpd
import xarray as xr
import fsspec
import numpy as np
import pyproj
import pandas as pd
from rechunker import rechunk

%matplotlib inline

## First, save a sample of NWM.

In [2]:
# Load the CHRTOUT data from the NWM Retrospective Zarr 2.1 dataset
# This has "Streamflow values at points associated with flow lines" 
# See https://registry.opendata.aws/nwm-archive/
nwm_uri = 's3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr'
ds = xr.open_zarr(fsspec.get_mapper(nwm_uri, anon=True, requester_pays=True))
ds

<xarray.Dataset>
Dimensions:     (feature_id: 2776738, time: 367439)
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
  * feature_id  (feature_id) int32 101 179 181 ... 1180001803 1180001804
    gage_id     (feature_id) |S15 dask.array<chunksize=(2776738,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(2776738,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Data variables:
    crs         |S1 ...
    streamflow  (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
    velocity    (time, feature_id) float64 dask.array<chunksize=(672, 30000), meta=np.ndarray>
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

In [ ]:
ds.streamflow.sel(time='05-20-2022:05-22-2022')

In [3]:
root_dir = '/opt/data/noaa/rechunker-test'
if os.path.isdir(root_dir):
    shutil.rmtree(root_dir)
makedirs(root_dir)

nwm_path = join(root_dir, 'nwm-sample')

In [4]:
ds.streamflow

<xarray.DataArray 'streamflow' (time: 367439, feature_id: 2776738)>
dask.array<open_dataset-97bd28df7335f5f3299f184f3cacc5destreamflow, shape=(367439, 2776738), dtype=float64, chunksize=(672, 30000), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
  * feature_id  (feature_id) int32 101 179 181 ... 1180001803 1180001804
    gage_id     (feature_id) |S15 dask.array<chunksize=(2776738,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(2776738,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(2776738,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 2020-12-31T23:0...
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

In [17]:
sub_ds = ds.isel(time=slice(0, 5000), feature_id=slice(0, 5000))

In [18]:
sub_ds.streamflow

<xarray.DataArray 'streamflow' (time: 5000, feature_id: 5000)>
dask.array<getitem, shape=(5000, 5000), dtype=float64, chunksize=(672, 5000), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
  * feature_id  (feature_id) int32 101 179 181 183 ... 194429 197191 197193
    gage_id     (feature_id) |S15 dask.array<chunksize=(5000,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(5000,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 1979-08-28T08:0...
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1

In [19]:
sub_ds.to_zarr(nwm_path)

In [31]:
target_chunks = {'feature_id': 300, 'time': 300}
# TODO what is max_mem for?
max_mem = '20MB'

target_store = join(root_dir, 'nwm-target.zarr')
temp_store = join(root_dir, 'nwm-temp.zarr')

# Note, if you get a ContainsArrayError, you probably need to delete temp_store and target_store first.
# See https://github.com/pangeo-data/rechunker/issues/78
rechunk_plan = rechunk(sub_ds, target_chunks, max_mem, target_store, temp_store=temp_store)
rechunk_plan

<Rechunked>
* Source      : <xarray.Dataset>
Dimensions:     (feature_id: 5000, time: 5000)
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
  * feature_id  (feature_id) int32 101 179 181 183 ... 194429 197191 197193
    gage_id     (feature_id) |S15 dask.array<chunksize=(5000,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(5000,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 1979-08-28T08:0...
Data variables:
    crs         object None
    streamflow  (time, feature_id) float64 dask.array<chunksize=(672, 5000), meta=np.ndarray>
    velocity    (time, feature_id) float64 dask.array<chunksize=(672, 5000), meta=np.ndarray>
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

* Intermediate: <zarr.hierarchy.Group '/'>

* Target      : <zarr.hierarchy.Group '/'>

In [32]:
rechunk_plan.execute()

<zarr.hierarchy.Group '/'>

In [33]:
rechunked_ds = xr.open_zarr(target_store)
rechunked_ds
# TODO why warning about consolidated metadata?

/tmp/ipykernel_10625/340056961.py:1: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  rechunked_ds = xr.open_zarr(target_store)


<xarray.Dataset>
Dimensions:     (feature_id: 5000, time: 5000)
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(300,), meta=np.ndarray>
  * feature_id  (feature_id) int32 101 179 181 183 ... 194429 197191 197193
    gage_id     (feature_id) |S15 dask.array<chunksize=(300,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(300,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(300,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(300,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 1979-08-28T08:0...
Data variables:
    crs         float64 ...
    streamflow  (time, feature_id) float64 dask.array<chunksize=(300, 300), meta=np.ndarray>
    velocity    (time, feature_id) float64 dask.array<chunksize=(300, 300), meta=np.ndarray>
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

In [34]:
rechunked_ds.streamflow

<xarray.DataArray 'streamflow' (time: 5000, feature_id: 5000)>
dask.array<open_dataset-d8875e79d45ac95f34d8893e2ee97460streamflow, shape=(5000, 5000), dtype=float64, chunksize=(300, 300), chunktype=numpy.ndarray>
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(300,), meta=np.ndarray>
  * feature_id  (feature_id) int32 101 179 181 183 ... 194429 197191 197193
    gage_id     (feature_id) |S15 dask.array<chunksize=(300,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(300,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(300,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(300,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1979-02-01T01:00:00 ... 1979-08-28T08:0...
Attributes:
    grid_mapping:  crs
    long_name:     River Flow
    units:         m3 s-1